In [56]:
version = "v1.6.092820"

---
# Assignment 1 Part 1: N-gram Language Models (40 pts)

In this assignment, we're going to train an n-gram language model that is able to "imitate" William Shakespeare's writing. 

In [57]:
# Configure nltk

import nltk

nltk_data_path = "assets/nltk_data"
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

In [101]:
from nltk import *

## Question 1: Load the dataset (10 pts)

As the first step towards imitating Shakespeare's writing, you will create a function called `load_data` that loads his original *Sonnets* from `assets/gutenberg/THE_SONNETS.txt`. This function should accomplish the following: 

* **Extract sentences from the data file.** Of course, depending on the nature of the task at hand, what constitutes a *sentence* can vary. In the context of this assignment, we will define a sentence as just a line of a sonnet, regardless of the punctuation at end. In addition, we will ignore the boundaries of the sonnets --- that is, we are not dealing with $154$ individual *sonnets* but rather $154 \times 14 = 2156$ *sentences* (actually only $2155$ sentences, as *Sonnet 99* has [15 lines](https://www.google.com/search?ei=po4hX9jzN4rKtQaL7K-wDg&q=why+is+sonnet+99+15+lines&oq=why+is+sonnet+99+15+lines&gs_lcp=CgZwc3ktYWIQAzoECAAQR1ChGlihGmCqHGgAcAJ4AIABXIgBXJIBATGYAQCgAQGqAQdnd3Mtd2l6wAEB&sclient=psy-ab&ved=0ahUKEwjY3snX3PLqAhUKZc0KHQv2C-YQ4dUDCAw&uact=5) but *Sonnet 126* has only [12](https://www.google.com/search?ei=d4whX6_uH9a4tQbUiISoDA&q=why+is+sonnet+126+only+12+lines&oq=o+thou+my+lovely+boy+who+in+thy+power&gs_lcp=CgZwc3ktYWIQARgBMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHUABYAGCWEWgAcAJ4AIABAIgBAJIBAJgBAKoBB2d3cy13aXrAAQE&sclient=psy-ab)). We encourage you to explore alternative definitions of a sentence on your own; for example, an entire sonnet could be modelled as a sentence. Finally, make sure that the newline character `\n` at the end of each line is dropped. 


* **Tokenise each extracted sentence.** While it's ambiguous what a sentence is, what constitutes a "*word*" is even more task-dependent. Do punctuations count as "words"? Are two "words" with the same spelling but different casing considered identical? Since what a text file contains is nothing more than a squence of characters, there are many possible ways of grouping these characters to form "words" that are subsequently taken as input by a program. To distinguish what's actually taken as input by a program from a linguistic word, we call the former a *token*. The process of producing a list of tokens out of a sentence is then called *tokenisation*. At this step, you will first lower-case each sentence extracted from the previous step entirely and then tokenise each lower-cased sentence. You may use any tokeniser of your choice, such as `word_tokenize` from the `nltk` library. The grading of the assignment doesn't depend on your choice of the tokeniser. 

**This function should return a `list` of length 2155, where each element is a `list` of `str` representing the tokens of each sentence as produced by the tokeniser of your choice. An example output would be:**

```
[['from', 'fairest', 'creatures', 'we', 'desire', 'increase', ','],
 ['that', 'thereby', 'beauty', '’', 's', 'rose', 'might', 'never', 'die', ','],
 ...
 ['came', 'there', 'for', 'cure', 'and', 'this', 'by', 'that', 'i', 'prove', ','], 
 ['love', '’', 's', 'fire', 'heats', 'water', ',', 'water', 'cools', 'not', 'love', '.']]
```

In [58]:
from nltk.tokenize import word_tokenize
def load_data():
    """
    Load text data from a file and produce a list of token lists
    """
    is_integer = lambda s: s.isdigit() or (s[0] == '-' and s[1:].isdigit())
    with open('assets/gutenberg/THE_SONNETS.txt', 'r') as f:
        sentences_no_newline = [line.strip() for line in f]
        sentences_no_empty_lines = list(filter(None, sentences_no_newline))
        sentences_no_integer = [item for item in sentences_no_empty_lines if not item.isdigit()]
        sentences_tokenized = [word_tokenize(i) for i in sentences_no_integer]
        sentences = [[j.lower() for j in i] for i in sentences_tokenized]
    
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return sentences

In [59]:
# Autograder test

stu_ans = load_data()

assert isinstance(stu_ans, list), "Q1: Your function should return a list. "
assert len(stu_ans) == 2155, "Q1: There should be 2155 sentences. "

for index, tokens in enumerate(stu_ans):
    assert isinstance(tokens, list), f"Q1: The element at index {index} of your answer list should be a list. "
    for token in tokens:
        if token.isalpha():
            assert token.islower(), f'Q1: Token "{token}" in the sentence at index {index} is not lower-cased. '
        
        assert token != "\n", f'Q1: You should drop the "\\n" character in the sentence at index {index}. '

del stu_ans

## Question 2: Build vocabulary (15 pts)

Next, we need a "vocabulary" that contains all the unique tokens. Moreover, as mentioned in the lecture, we often pad a sentence with `<s>` and `</s>` to indicate its start and end when working with n-gram language models; therefore, these two special tokens should also be included in our vocabulary. Complete the function below to build a vocabulary. The order in which the tokens are stored doesn't matter. 

**This function should return a `list` of unique tokens, including `<s>` and `</s>`. An example output would be:**

```
['refuse',
 'enjoyed',
 ...
 '<s>', 
 '</s>']
```

In [60]:
from itertools import chain

def build_vocab(sentences):
    """
    Take a list of sentences and return a vocab
    """
    
    list_of_sentences = sentences
    
    vocab_without_specialtoken = list(set(chain(*list_of_sentences)))
    vocab_without_specialtoken.extend(['<s>', '</s>' ])
    vocab = vocab_without_specialtoken                                   
    
   
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return vocab

In [61]:
# Autograder tests

stu_sents = load_data()
stu_vocab = build_vocab(stu_sents)

assert isinstance(stu_vocab, list), "Q2: Your function should return a list. "
assert stu_vocab, "Q2: Your vocab is empty. "
assert "<s>" in stu_vocab, "Q2: Remember to include special token <s>. "
assert "</s>" in stu_vocab, "Q2: Remember to include special token </s>. "

# Some hidden tests

del stu_sents, stu_vocab

## Question 3: Generate all $n$-grams (15 pts)

Now let's write a function to generate all $n$-grams for each sentence. This can be accomplished in two steps:
* **Pad each sentence with `<s>` and `</s>` for $n \geq 2$**. You need $n - 1$ paddings on both ends of a sentence, so that there are two $n$-grams that model the first and the last token, respectively. You may implement the padding function yourself or use the `pad_both_ends` function from the `nltk` library. 


* **Generate $n$-grams on the padded sentences**. Check out the `ngrams` function from `nltk`. For a sentence with $\ell$ tokens excluding paddings, the maximum number of possible $n$-grams generated from its padded version should be $\ell + n - 1$. Think about why. 

**Complete the function below to return a `list`, where each element of the `list` is a either a list or a "generator object" produced by the `ngrams` function, representing a sequence of all $n$-grams generated from each appropriately padded sentence. If the argument `n=2`, the autograder would accept either of the example outputs below:**

```
[<generator object ngrams at 0x7f77ed778b10>,
 <generator object ngrams at 0x7f77e7d934f8>,
 ...
 <generator object ngrams at 0x7f77e7d751b0>,
 <generator object ngrams at 0x7f77e7d75228>]
```

**OR**

```
[
    [('<s>', 'from'), ('from', 'fairest'), ('fairest', 'creatures'), ('creatures', 'we'), ('we', 'desire'),
     ('desire', 'increase'), ('increase', ','), (',', '</s>')], 
    
    [('<s>', 'that'), ('that', 'thereby'), ('thereby', 'beauty'), ('beauty', '’'), ('’', 's'), ('s', 'rose'), 
     ('rose', 'might'), ('might', 'never'), ('never', 'die'), ('die', ','), (',', '</s>')],
     
    ...
    
    [('<s>', 'came'), ('came', 'there'), ('there', 'for'), ('for', 'cure'), ('cure', 'and'), ('and', 'this'), 
     ('this', 'by'), ('by', 'that'), ('that', 'i'), ('i', 'prove'), ('prove', ','), (',', '</s>')], 
    
    [('<s>', 'love'), ('love', '’'), ('’', 's'), ('s', 'fire'), ('fire', 'heats'), ('heats', 'water'), 
     ('water', ','), (',', 'water'), ('water', 'cools'), ('cools', 'not'), ('not', 'love'), ('love', '.'), 
     ('.', '</s>')]
]
```

In [128]:
from nltk import ngrams
from nltk.lm.preprocessing import pad_both_ends
def build_ngrams(n, sentences):
    """
    Take a list of unpadded sentences and create all n-grams as specified by the argument "n" for each sentence
    """
    updated_sentences = []
    all_ngrams = []
    

    for sentence in sentences:
        sentence = list(pad_both_ends(sentence,  n=n))
        updated_sentences.append(sentence)
        
        

        
    for sentence in updated_sentences:
        sentence = list(ngrams(sentence, n))
        all_ngrams.append(sentence)
        
    
    
    return list(all_ngrams)

    


In [129]:
# Autograder tests

stu_n = 4
stu_sents = load_data()
stu_ngrams = build_ngrams(stu_n, stu_sents)

assert isinstance(stu_ngrams, list), "Q3: Your function should return a list. "
assert stu_ngrams, "Q3: Your ngrams list is empty. "

# Some hidden tests


del stu_n, stu_sents, stu_ngrams

Now that we have completed all the preparation work for imitating William Shakespeare's writing, it's time to take a break. We will resume in Assignment 1 Part 2 to finish training an $n$-gram language model. See you there!